In [25]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [26]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path = 'cmn.txt'

In [27]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding ='utf8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [28]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [29]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2623
Max sequence length for inputs: 30
Max sequence length for outputs: 22


In [30]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [31]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [32]:
print('encoder_input_data shape', encoder_input_data.shape)
print('decoder_input_data shape', decoder_input_data.shape)
print('decoder_target_data shape ', decoder_target_data.shape)

encoder_input_data shape (10000, 30, 73)
decoder_input_data shape (10000, 22, 2623)
decoder_target_data shape  (10000, 22, 2623)


In [33]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):        
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [34]:
# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [35]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [36]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 13s 2ms/step - loss: 2.0301 - val_loss: 2.4903
Epoch 2/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.9066 - val_loss: 2.4339
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.8051 - val_loss: 2.3132
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.7161 - val_loss: 2.2470
Epoch 5/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.6342 - val_loss: 2.1687
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.5612 - val_loss: 2.1023
Epoch 7/100
8000/8000 [==============================] - 12s 1ms/step - loss: 1.4936 - val_loss: 2.0408
Epoch 8/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.4285 - val_loss: 1.9952
Epoch 9/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.3741 - val_loss: 1.9446
Epoch 10/100
800

In [44]:
model.save('s2s.h5')

/opt/conda/lib/python3.6/site-packages/keras/engine/topology.py:2361: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [45]:
# inference
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [46]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

In [47]:
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [48]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [59]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
       
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
#     print("output_tokens", output_tokens)
#     print("h", h)
#     print("c", c)
    return decoded_sentence

In [106]:
def translate(word):
    inputs =  encoder_input_data[input_texts.index(word): input_texts.index(word) + 1]
    decoded_sentence = decode_sequence(inputs)
    print('result :', decoded_sentence)

In [108]:
translate("Wait!")

Input sentence: 等等！



In [100]:
for seq_index in range(5):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*', input_seq.shape, seq_index)
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])

* (1, 30, 73) 0
Input sentence: Hi.
Decoded sentence: 你好。

* (1, 30, 73) 1
Input sentence: Hi.
Decoded sentence: 你好。

* (1, 30, 73) 2
Input sentence: Run.
Decoded sentence: 你用跑的。

* (1, 30, 73) 3
Input sentence: Wait!
Decoded sentence: 等等！

* (1, 30, 73) 4
Input sentence: Hello!
Decoded sentence: 你好。

